In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!pip install biopython

In [2]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from Bio.SeqIO import parse
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from sklearn.preprocessing import LabelEncoder
import re
import keras
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, LeakyReLU
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from collections import Counter
from sklearn.utils import class_weight
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


In [3]:
df = pd.read_csv('/content/drive/My Drive/USA_removed.csv')

In [4]:
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^acgt]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array

In [5]:
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','z']))

LabelEncoder()

In [6]:
def ordinal_encoder(my_array):
    
    integer_encoded = label_encoder.transform(my_array)
    float_encoded = integer_encoded.astype(float)
    float_encoded[float_encoded == 0] = 0.25 # A
    float_encoded[float_encoded == 1] = 0.50 # C
    float_encoded[float_encoded == 2] = 0.75 # G
    float_encoded[float_encoded == 3] = 1.00 # T
    float_encoded[float_encoded == 4] = 0.00 # anything else, z
    
    return float_encoded

In [7]:
data=df[["Sequence","Geo_Location"]]
data=data[data["Sequence"].notna()]

dummy=[]
dum=np.array(dummy)

form={"inp_seq":dum}

seq_df = pd.DataFrame (form, columns = ['inp_seq'])

seq_list=[]

for idx, seq in enumerate(list(data["Sequence"])):
    arr=ordinal_encoder(string_to_array(seq))
    seq_list.append(arr)
    
seq_df["inp_seq"]=seq_list

final_data= data.assign(enc_seq=seq_df)

final_data=final_data[["enc_seq","Geo_Location"]]

final_data

,enc_seq,Geo_Location
0,"[0.5, 0.5, 0.25, 0.25, 0.5, 1.0, 1.0, 1.0, 0.5...",Virginia
1,"[1.0, 0.5, 0.5, 0.5, 0.25, 0.75, 0.75, 1.0, 0....",Virginia
2,"[0.25, 0.5, 0.5, 0.25, 0.25, 0.5, 0.5, 0.25, 0...",Virginia
3,"[0.25, 0.5, 1.0, 1.0, 1.0, 0.5, 0.75, 0.25, 1....",Virginia
4,"[0.25, 0.25, 0.25, 0.75, 0.75, 1.0, 1.0, 1.0, ...",Virginia
...,...,...
4479,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",California
4480,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",Washington
4481,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",Washington
4482,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",California


In [8]:
def get_maxLen(enc_seq):
    
    max=0
    for row in enc_seq:
        #print(type(row))
        if(len(row)>max):
            max=len(row)
    
    
    return max

In [9]:
def append_arr(enc_seq, max_len):
    
    seq_l=list(enc_seq)
    for i in range(len(seq_l),max_len):
        seq_l.append(0)
        
    new_seq_ar=np.array(seq_l)

        
    return new_seq_ar

In [10]:
import math

max_len=get_maxLen(final_data["enc_seq"])
print("max_len is",max_len)
if max_len%2 !=0:
    max_len += 1

n = 1
for i in range(1, math.floor(math.sqrt(max_len))):
    if max_len%i==0:
        n = i

padded_seq_list=[]

for index, row in final_data.iterrows():
    seq_ar=append_arr(row["enc_seq"],max_len)
    padded_seq_list.append(seq_ar)

dummy=[]
dum=np.array(dummy)
form={"padded_enc_seq":dum}
padded_seq_df = pd.DataFrame (form, columns = ['padded_enc_seq'])

padded_seq_df["padded_enc_seq"]=padded_seq_list

padded_final_data= final_data.assign(padded_enc_seq=padded_seq_df)

padded_final_data=padded_final_data[["padded_enc_seq","Geo_Location"]]

padded_final_data.head(5)

max_len is 29921


,padded_enc_seq,Geo_Location
0,"[0.5, 0.5, 0.25, 0.25, 0.5, 1.0, 1.0, 1.0, 0.5...",Virginia
1,"[1.0, 0.5, 0.5, 0.5, 0.25, 0.75, 0.75, 1.0, 0....",Virginia
2,"[0.25, 0.5, 0.5, 0.25, 0.25, 0.5, 0.5, 0.25, 0...",Virginia
3,"[0.25, 0.5, 1.0, 1.0, 1.0, 0.5, 0.75, 0.25, 1....",Virginia
4,"[0.25, 0.25, 0.25, 0.75, 0.75, 1.0, 1.0, 1.0, ...",Virginia


In [11]:
def reshape_seq(seq, m ,n):
    
    r_seq=np.reshape(seq,(m,n))
    
    return r_seq

re_seqList=[]

for index, row in padded_final_data.iterrows():
    seq_ar=reshape_seq(row["padded_enc_seq"],n,max_len//n)
    re_seqList.append(seq_ar)

In [12]:
X = np.asarray(re_seqList)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2], 1))
X.shape

(4484, 6, 4987, 1)

In [13]:
Y = padded_final_data['Geo_Location'].values
Y.shape

(4484,)

In [14]:
loc_classes = list(Y)
loc_classes = np.array(loc_classes) 
loc_classes = list(np.unique(loc_classes))

In [15]:
df = padded_final_data['Geo_Location'].apply(loc_classes.index)
Y = df.values
Y.shape

(4484,)

In [16]:
!pip install keras-metrics

In [17]:
import keras
import keras_metrics
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, LeakyReLU
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from collections import Counter
from sklearn.utils import class_weight
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
import tensorflow as tf

In [18]:
#x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.30)
acc_per_fold = []
loss_per_fold = []
num_folds = 3 # there will be 3 folds in it's entirety
kfold = KFold(n_splits=num_folds, shuffle=True)

In [19]:
# print("Train X Shape:",x_train.shape)
# print("Train Y Shape:",y_train.shape)
# print("Validation X Shape:",x_val.shape)
# print("Validation Y Shape:",y_val.shape)
print(X.shape)

(4484, 6, 4987, 1)


In [20]:
# batch_size = 64
# num_classes = 8
# epochs = 10

#img_rows, img_cols = x_train.shape[1], x_train.shape[2]
img_rows, img_cols = X.shape[1],X.shape[2]
# x_train = x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
# x_val = x_val.reshape(x_val.shape[0],img_rows,img_cols,1)
# print(y_train.shape)

# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_val.shape[0], 'validation samples')

# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_val = keras.utils.to_categorical(y_val, num_classes)
Y = keras.utils.to_categorical(Y,8)

In [21]:
fold_no = 1
num_classes = 8
for train,test in kfold.split(X,Y):
  model = Sequential()
  model.add(Conv2D(16, kernel_size=(3,3), activation='relu', input_shape=(img_rows,img_cols,1)))
  model.add(Conv2D(16, (3, 3),activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, (3,3), padding='same',activation='relu'))
  model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(32, activation='relu'))
  model.add(Dense(16, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))
  # compilation of model
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
  print('------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  model.fit(X[train],Y[train], batch_size = 64, epochs = 200,verbose = 1, validation_data= (X[test],Y[test]))
  scores = model.evaluate(X[test],Y[test],verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])
  fold_no = fold_no + 1

------------------------------------------------------------
Training for fold 1 ...
Train on 2989 samples, validate on 1495 samples
Epoch 1/200
2989/2989 [==============================] - 10s 3ms/step - loss: 1.9878 - accuracy: 0.3175 - val_loss: 1.6638 - val_accuracy: 0.4923
Epoch 2/200
2989/2989 [==============================] - 6s 2ms/step - loss: 1.6707 - accuracy: 0.4015 - val_loss: 1.4599 - val_accuracy: 0.5130
Epoch 3/200
2989/2989 [==============================] - 6s 2ms/step - loss: 1.5745 - accuracy: 0.4527 - val_loss: 1.2911 - val_accuracy: 0.5532
Epoch 4/200
2989/2989 [==============================] - 6s 2ms/step - loss: 1.4960 - accuracy: 0.4968 - val_loss: 1.2366 - val_accuracy: 0.5645
Epoch 5/200
2989/2989 [==============================] - 6s 2ms/step - loss: 1.4312 - accuracy: 0.5256 - val_loss: 1.2502 - val_accuracy: 0.5960
Epoch 6/200
2989/2989 [==============================] - 6s 2ms/step - loss: 1.3884 - accuracy: 0.5453 - val_loss: 1.1532 - val_accuracy: 0.6

In [22]:
print('------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('-----------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('--------------------------------------------------')
print('Average score for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)}')
print(f'>Loss: {np.mean(loss_per_fold)}')
#print(model.summary())

------------------------------------------------
Score per fold
-----------------------------------------------------
> Fold 1 - Loss: 1.1561970226342064 - Accuracy: 79.3311059474945%
-----------------------------------------------------
> Fold 2 - Loss: 1.0956845111073459 - Accuracy: 76.92307829856873%
-----------------------------------------------------
> Fold 3 - Loss: 1.9608078734223622 - Accuracy: 79.78581190109253%
--------------------------------------------------
Average score for all folds:
> Accuracy: 78.67999871571858
>Loss: 1.4042298023879713
